# 🤖 日本語・奇妙生成AI (ChaosAI-JP v2)

このノートブックでは、`rinna/japanese-gpt2-small` を使って、
奇妙な日本語文（50文）をLoRAで学習し、生成することができます。

In [ ]:
# ✅ 必要なライブラリをインストール
!pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
# ✅ モデルとトークナイザーの読み込み
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'rinna/japanese-gpt2-small'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')

In [ ]:
# ✅ 学習用データ（50の奇妙な日本語文）
train_data = [
    {"text": "冷蔵庫が笑いながら哲学を吐いた。"},    {"text": "昨日の空は、味噌汁に似ていたと思う。"},    {"text": "私は靴下の夢を見て、現実に起きなかった。"},    {"text": "時間が逆立ちして、火曜日が二度死んだ。"},    {"text": "猫は存在論を唱え、静かに消えた。"},    {"text": "机が恋をしたのは、消しゴムの亡霊だった。"},    {"text": "電話の鳴き声が、夜を赤く染めた。"},    {"text": "目覚まし時計が眠れなくて泣いていた。"},    {"text": "空気が記憶を取り戻してしまった午後。"},    {"text": "石鹸が国家について討論していた。"},    {"text": "今日はペンギンが風になる日だった。"},    {"text": "雲の一部が秘密を漏らしてしまった。"},    {"text": "水が燃え、火が凍る町に住んでいた。"},    {"text": "傘が開かなくても雨は歌い続けた。"},    {"text": "図書館が私の名前を呟いた気がする。"},    {"text": "カーテンが自分の影に怯えていた。"},    {"text": "卵が進化論について疑問を抱いた。"},    {"text": "文字たちが逃げ出したノートを開いた。"},    {"text": "リモコンが遠隔操作されて反逆した。"},    {"text": "人間のふりをした机が一人で立ち上がった。"},    {"text": "正午にしか現れない影が時計を盗んだ。"},    {"text": "おにぎりが戦争を止めようとしていた。"},    {"text": "時計の針が自分の存在に疑問を持った。"},    {"text": "タンスが昔の記憶を語り始めた。"},    {"text": "道路が夢を見たと告白してきた。"},    {"text": "植物がインターネットに接続された。"},    {"text": "ひとりごとが独り歩きして遠くへ行った。"},    {"text": "自動販売機が詩を朗読していた。"},    {"text": "布団が恋人を待っていた夜の話。"},    {"text": "電子レンジの中で時間が溶けた。"},    {"text": "風邪が風と喧嘩して負けたらしい。"},    {"text": "ドアノブが自己紹介を始めたとき、僕は目を覚ました。"},    {"text": "言葉が腐って、音だけが残った。"},    {"text": "朝ごはんが逃げ出したので、昼まで空腹だった。"},    {"text": "影が本体を置き去りにして旅に出た。"},    {"text": "靴が空を飛んだ午後三時の空気。"},    {"text": "鉛筆が小説を書きたがっていた。"},    {"text": "今日のカーテンは何か企んでいる顔をしていた。"},    {"text": "公園のベンチが私の過去を知っていた。"},    {"text": "ネジが社会について怒っていた。"},    {"text": "音が時間の奥から逆走してきた。"},    {"text": "月が笑った夜、犬が喋った。"},    {"text": "牛乳が記憶を持ったまま腐っていた。"},    {"text": "爪切りが泣いていたのは夢の中だけだった。"},    {"text": "ドーナツに穴がないことに絶望していた。"},    {"text": "シャツが脱ぎ捨てられる瞬間、叫んだ。"},    {"text": "床が浮かんでいる理由は誰にもわからなかった。"},    {"text": "コップが孤独をテーマに演説した。"},    {"text": "アイスクリームが未来を予言したが誰も信じなかった。"},    {"text": "ラジオが自分の声に怯えていた。"},]
from datasets import Dataset
dataset = Dataset.from_list(train_data)

In [ ]:
# ✅ LoRA学習設定と実行
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=32, lora_dropout=0.1, bias="none"
)
model = get_peft_model(model, lora_config)

def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=64)

tokenized_dataset = dataset.map(tokenize)

training_args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=1,
    save_steps=10,
    save_total_limit=1,
    fp16=True,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# ✅ 生成テスト（設定つき）
input_text = "冷蔵庫が"
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    temperature=1.2,
    top_k=50,
    repetition_penalty=1.5
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))